In [0]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [0]:
url="https://www.worldometers.info/coronavirus/"

In [83]:
response = requests.get(url)
print(response)

<Response [200]>


In [0]:
soup = BeautifulSoup(response.content,'html.parser')
#print(soup.prettify())

In [0]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'


In [0]:
class Format:
    END = '\033[0m'
    UNDERLINE = '\033[4m'

In [87]:
def getTitle():
  return soup.title

getTitle()

<title>Coronavirus Update (Live): 3,308,901 Cases and 234,133 Deaths from COVID-19 Virus Pandemic - Worldometer</title>

In [88]:
def getTotal_Cases():
  worldwide = soup.find_all(id="maincounter-wrap")

  print (color.BOLD + Format.UNDERLINE + 'COVID-19 CORONAVIRUS WORLDWIDE DATA' + Format.END + color.END,end='\n' *2)

  for data in worldwide:
    h1Data = data.find('h1')
    spanData = data.find('span')

    if None in (h1Data,spanData):
      continue

    print(h1Data.text.strip() + ' ' + spanData.text.strip())
    print(end='\n')


getTotal_Cases()

COVID-19 CORONAVIRUS WORLDWIDE DATA

Coronavirus Cases: 3,308,901

Deaths: 234,133

Recovered: 1,042,993



In [0]:
def getMainValue(cases):
  if (cases.select('div')[0] and cases.select('div')[1]):
    caseText1 = cases.select('div')[1].text.strip()
    caseValue1 = cases.select('div')[0].text.strip()
    print (caseText1 + ': ' + color.BOLD + caseValue1 + color.END,end='\n' *2)

def getLeftSideValue(cases):
  if (cases.select('div')[3]):
    caseText2 = (cases.select('div')[3]).select('div')
    caseValue2 = (cases.select('div')[3]).find("span", class_="number-table")
    caseStrongValue2 = (cases.select('div')[3]).find("strong")
    print (caseText2[0].text.strip() + ': ' + color.BOLD + color.BLUE + caseValue2.text.strip() + "(" + caseStrongValue2.text.strip()+ "%)" +color.END,end='\n' *2)

def getRightSideValue(cases):
  if (cases.select('div')[5]):  
    caseText3 = (cases.select('div')[5]).select('div')
    caseValue3 = (cases.select('div')[5]).find("span", class_="number-table")
    caseStrongValue3 = (cases.select('div')[5]).find("strong")
    print (caseText3[0].text.strip() + ': ' + color.BOLD + color.RED + caseValue3.text.strip() + "(" + caseStrongValue3.text.strip() + "%)" +color.END,end='\n' *2)
  


In [90]:
def getActive_Cases():
  activeCases=soup.find("div", class_="panel_flip")

  print (color.BOLD + Format.UNDERLINE + 'ACTIVE CASES' + Format.END + color.END,end='\n' *2)

  activeData = activeCases.find("div", class_="panel_front")

  if not activeData:
    return
  
  if not activeData.select('div'):
    return
 
  #get Currently Infected Patients and its value
  getMainValue(activeData)
  
  #get mild condition and its value
  getLeftSideValue(activeData)

  #get critical condition and its value
  getRightSideValue(activeData)
 

getActive_Cases()

ACTIVE CASES

Currently Infected Patients: 2,031,775

in Mild Condition: 1,980,838(97%)

Serious or Critical: 50,937(3%)



In [91]:
def getClosed_Cases():
  
  print (color.BOLD + Format.UNDERLINE + 'CLOSED CASES' + Format.END + color.END,end='\n' *2)

  if (soup.find_all("div", class_="panel_flip") and soup.find_all("div", class_="panel_flip")[1]):
    closedCases=soup.find_all("div", class_="panel_flip")[1].select('div.panel_front')

  if (not closedCases[0]):
    return
 
  #get Outcome Cases and its value
  getMainValue(closedCases[0])

  #get recovered/discharged cases and its value
  getLeftSideValue(closedCases[0])

  #get death cases and its value
  getRightSideValue(closedCases[0])

  
getClosed_Cases()

CLOSED CASES

Cases which had an outcome:: 1,277,126

Recovered / Discharged: 1,042,993(82%)

Deaths: 234,133(18%)



In [92]:
def getOverallData():
  
  coronatable=soup.find("table")

  country=[]
  total_cases=[]
  new_cases=[]
  total_deaths=[]
  new_deaths=[]
  total_recovered=[]
  active_cases=[]
  serious_cases=[]

  rows=coronatable.find_all("tr")[9:-8]

  for row in rows:
    col=row.find_all("td")
    country.append(col[0].text.strip())
    total_cases.append(col[1].text.strip())
    new_cases.append(col[2].text.strip())
    total_deaths.append(col[3].text.strip())
    new_deaths.append(col[4].text.strip())
    total_recovered.append(col[5].text.strip())
    active_cases.append(col[6].text.strip())
    serious_cases.append(col[7].text.strip())


  df=pd.DataFrame(list(zip(country, total_cases,new_cases,total_deaths,new_deaths,total_recovered,active_cases,serious_cases)), 
                  columns=["Country","Total_Cases","New_Cases","Total_Deaths","New_Deaths","Total_recovered","Active_cases","Serious/Critical_cases"])

  df.replace(r'^\s*$', np.nan, regex=True, inplace=True)
  df.replace(np.nan,0, inplace=True)
  print(df.head(50))


getOverallData()




               Country Total_Cases  ... Active_cases Serious/Critical_cases
0                  USA   1,095,304  ...      875,696                 15,226
1                Spain     239,639  ...       77,112                  2,676
2                Italy     205,463  ...      101,551                  1,694
3                   UK     171,253  ...      144,138                  1,559
4               France     167,178  ...       93,326                  4,019
5              Germany     163,009  ...       32,886                  2,415
6               Turkey     120,204  ...       68,144                  1,514
7               Russia     106,498  ...       93,806                  2,300
8                 Iran      94,640  ...       13,509                  2,976
9               Brazil      87,187  ...       45,246                  8,318
10              Canada      53,236  ...       28,629                    557
11             Belgium      48,519  ...       29,349                    769
12         N

In [93]:
bs=BeautifulSoup(response.content, "lxml")
table_body=bs.find('tbody')
rows = table_body.find_all('tr')[8:-8]
allData = []
for row in rows:
  cols=row.find_all('td')
  cols=[x.text.strip() for x in cols]
  allData.append(cols)

#df1 = pd.DataFrame(data=allData, columns= ["Country","Total_Cases","New_Cases","Total_Deaths","New_Deaths","Total_recovered","Active_cases","Serious/Critical_cases","c9","c10","c11","c12","c13"])

df1 = pd.DataFrame(list(zip(allData[0],allData[1],allData[2],allData[3],allData[4])), columns= ["Country","Total_Cases","New_Cases","Total_Deaths","New_Deaths"])
print(df1.head(10))

     Country Total_Cases New_Cases Total_Deaths New_Deaths
0        USA       Spain     Italy           UK     France
1  1,095,304     239,639   205,463      171,253    167,178
2       +281                                              
3     63,871      24,543    27,967       26,771     24,376
4        +15                                              
5    155,737     137,984    75,945          N/A     49,476
6    875,696      77,112   101,551      144,138     93,326
7     15,226       2,676     1,694        1,559      4,019
8      3,309       5,125     3,398        2,523      2,561
9        193         525       463          394        373


**Indian** **Data**

In [94]:
allData1 = []
for row in rows:
  cols=row.find_all('td')
  for x in cols:
    if x.text.strip() == 'India':
      allData1.append(cols)

print(allData1[0])



[<td style="font-weight: bold; font-size:15px; text-align:left;"><a class="mt_a" href="country/india/">India</a></td>, <td style="font-weight: bold; text-align:right">35,043</td>, <td style="font-weight: bold; text-align:right;background-color:#FFEEAA;">+180</td>, <td style="font-weight: bold; text-align:right;">1,154 </td>, <td style="font-weight: bold; 
                                    text-align:right;"></td>, <td style="font-weight: bold; text-align:right">9,068</td>, <td style="text-align:right;font-weight:bold;">24,821</td>, <td style="font-weight: bold; text-align:right"></td>, <td style="font-weight: bold; text-align:right">25</td>, <td style="font-weight: bold; text-align:right">0.8</td>, <td style="font-weight: bold; text-align:right">902,654</td>, <td style="font-weight: bold; text-align:right">654</td>, <td data-continent="Asia" style="display:none">Asia</td>]


In [95]:
def getIndianData():
  
  coronatable_all=soup.find("table")

  country=[]
  total_cases=[]
  new_cases=[]
  total_deaths=[]
  new_deaths=[]
  total_recovered=[]
  active_cases=[]
  serious_cases=[]

  rows=coronatable_all.find_all("tr")[9:-8]

  for row in rows:
    col=row.find_all("td")
    if col[0].text.strip() == 'India':
      country.append(col[0].text.strip())
      total_cases.append(col[1].text.strip())
      new_cases.append(col[2].text.strip())
      total_deaths.append(col[3].text.strip())
      new_deaths.append(col[4].text.strip())
      total_recovered.append(col[5].text.strip())
      active_cases.append(col[6].text.strip())
      serious_cases.append(col[7].text.strip())


  df=pd.DataFrame(list(zip(country, total_cases,new_cases,total_deaths,new_deaths,total_recovered,active_cases,serious_cases)), 
                  columns=["Country","Total_Cases","New_Cases","Total_Deaths","New_Deaths","Total_recovered","Active_cases","Serious/Critical_cases"])

  df.replace(r'^\s*$', np.nan, regex=True, inplace=True)
  df.replace(np.nan,0, inplace=True)
  print(df.head(50))

  
getIndianData()

  Country Total_Cases  ... Active_cases Serious/Critical_cases
0   India      35,043  ...       24,821                    0.0

[1 rows x 8 columns]


In [0]:
indiaUrl="https://www.worldometers.info/coronavirus/country/india/"

resp = requests.get(indiaUrl)
print(resp)

soup = BeautifulSoup(resp.content,'html.parser')
print(soup.prettify())

In [97]:
def getTotal_Cases_for_India():
  worldwide = soup.find_all(id="maincounter-wrap")

  print (color.BOLD + Format.UNDERLINE + 'COVID-19 CORONAVIRUS INDIAN DATA' + Format.END + color.END,end='\n' *2)

  for data in worldwide:
    h1Data = data.find('h1')
    spanData = data.find('span')

    if None in (h1Data,spanData):
      continue

    print(h1Data.text.strip() + ' ' + spanData.text.strip())
    print(end='\n')


getTotal_Cases_for_India()

COVID-19 CORONAVIRUS INDIAN DATA

Coronavirus Cases: 35,043

Deaths: 1,154

Recovered: 9,068



In [99]:
def getClosed_Cases_for_India():
  closedIndianCases=soup.find("div", class_="panel_flip")

  print (color.BOLD + Format.UNDERLINE + 'CLOSED CASES' + Format.END + color.END,end='\n' *2)

  closedIndianData = closedIndianCases.find("div", class_="panel_front")

  if not closedIndianData:
    return
  
  if not closedIndianData.select('div'):
    return
 
  #get Currently Infected Patients and its value
  getMainValue(closedIndianData)
  
  #get mild condition and its value
  getLeftSideValue(closedIndianData)

  #get critical condition and its value
  getRightSideValue(closedIndianData)
 

getClosed_Cases_for_India()

CLOSED CASES

Cases which had an outcome:: 10,222

Recovered / Discharged: 9,068(89%)

Deaths: 1,154(11%)

